In [1]:
# Dataset
# https://drive.google.com/file/d/192jeGRTCZZfet8ufHPfaMn05T7Biklfw/view?usp=sharing

In [2]:
from tensorflow.keras.preprocessing import text_dataset_from_directory

# Assumes you're in the root level of the dataset directory.
# If you aren't, you'll need to change the relative paths here.
train_data = text_dataset_from_directory("movie-reviews-dataset/movie-reviews-dataset/test")
test_data = text_dataset_from_directory("movie-reviews-dataset/movie-reviews-dataset/train")

Found 25000 files belonging to 2 classes.


2022-11-15 22:59:29.448609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-15 22:59:29.473081: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-15 22:59:29.473220: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mukesh-Inspiron-5468): /proc/driver/nvidia/version does not exist
2022-11-15 22:59:29.629661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 25000 files belonging to 2 classes.


In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace

def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [4]:
train_data = prepareData("movie-reviews-dataset/movie-reviews-dataset/test")
test_data = prepareData("movie-reviews-dataset/movie-reviews-dataset/train")

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
for text_batch, label_batch in train_data.take(1):
    print(text_batch.numpy()[0])
    print(label_batch.numpy()[0]) # 0 = negative, 1 = positive

b'Symbolism galore, great tunes, this film crushed their "soon to be no more" target audience\'s expectations. These monkees and the naturally selected members of the group, were witnessing a subtle yet in your face, kiss goodbye to each other. The message rings true today, the cage you escape from and the bridge you want to jump off of, are the next generations own disappointments, there will always be new kids on the block replacing those who break free from the chains. The film can be frustrating at times, because the themes the film attacks are so blatantly apart of the American way of life, a thinking and reasoning person cannot help but stare at their own reflection in the scenes of Head, and question not only their personal motives for continuing the madness of everyday American life, but the motives of those who want it to continue for the sake of madness. The final scene, similar to Don Quixote\'s chivalric daring of the caged tiger to exit for battle, represents just how delu

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

In [7]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  # Max vocab size. Any words outside of the max_tokens most common ones
  # will be treated the same way: as "out of vocabulary" (OOV) tokens.
  max_tokens=max_tokens,
  # Output integer indices, one per string token
  output_mode="int",
  # Always pad or truncate to exactly this many tokens
  output_sequence_length=max_len,
)

In [8]:
# Call adapt(), which fits the TextVectorization layer to our text dataset.
# This is when the max_tokens most common words (i.e. the vocabulary) are selected.
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

In [9]:
model.add(vectorize_layer)

In [10]:
from tensorflow.keras.layers import Embedding
model.add(Embedding(max_tokens + 1, 128))

In [11]:
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
model.add(LSTM(64))

In [12]:
model.add(Embedding(max_tokens + 1, 128))

# ----- 4. RECURRENT LAYER
model.add(LSTM(64))

# ----- 5. DENSE HIDDEN LAYER
model.add(Dense(64, activation="relu"))

# ----- 6. OUTPUT
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 128)          128128    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 embedding_1 (Embedding)     (None, 64, 128)           128128    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [14]:
# from keras.layers import SimpleRNN
# # build model
# model.add(SimpleRNN(128, return_sequences=True))
# # model.add(SimpleRNN(128, return_sequences=True))
# model.add(SimpleRNN(128, return_sequences=False))
# model.add(Dense(20))
# model.add(Dense(64, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))
# model.build()
# model.summary()

In [15]:
# model.add(Dense(64, activation="relu"))

In [16]:
# model.add(Dense(1, activation="sigmoid"))

In [13]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [14]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 41s 46ms/step - loss: 0.6935 - accuracy: 0.5025
Epoch 2/10
782/782 [==============================] - 38s 48ms/step - loss: 0.6933 - accuracy: 0.4987
Epoch 3/10
782/782 [==============================] - 38s 48ms/step - loss: 0.6933 - accuracy: 0.4998
Epoch 4/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6933 - accuracy: 0.4990
Epoch 5/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6933 - accuracy: 0.4996
Epoch 6/10
782/782 [==============================] - 37s 48ms/step - loss: 0.6932 - accuracy: 0.4982
Epoch 7/10
782/782 [==============================] - 38s 48ms/step - loss: 0.6932 - accuracy: 0.5010
Epoch 8/10
782/782 [==============================] - 35s 45ms/step - loss: 0.6932 - accuracy: 0.4984
Epoch 9/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6932 - accuracy: 0.4995
Epoch 10/10
782/782 [==============================] - 36s 45ms/step - loss: 0.693

In [15]:
model.save('movie-reviews-dataset/movie-reviews-dataset')

INFO:tensorflow:Assets written to: movie-reviews-dataset/movie-reviews-dataset/assets


INFO:tensorflow:Assets written to: movie-reviews-dataset/movie-reviews-dataset/assets


In [16]:
from tensorflow import keras
model = keras.models.load_model('movie-reviews-dataset/movie-reviews-dataset')

In [17]:
# Should print a very high score like 0.98.
print(model.predict([
  "i loved it! highly recommend it to anyone and everyone looking for a great movie to watch.",
]))

# Should print a very low score like 0.01.
print(model.predict([
  "this was awful! i hated it so much, nobody should watch this. the acting was terrible, the music was terrible, overall it was just bad.",
]))

1/1 [==============================] - 1s 1s/step
[[0.50448453]]
1/1 [==============================] - 0s 43ms/step
[[0.50448453]]
